In [1]:
import tensorflow as tf
import pickle
import numpy as np
import csv

In [2]:
tf.zeros(1)
NB_SPEAKERS = 251

2022-01-09 23:49:40.427808: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-09 23:49:40.824722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3939 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:0a:00.0, compute capability: 7.5


In [3]:
x_train, y_train = pickle.load(open('dataset.p', 'rb'))
dim = len(x_train[0])
y_train = tf.keras.utils.to_categorical(y_train, num_classes=NB_SPEAKERS)
y_train = tf.convert_to_tensor(y_train)
x_train = tf.convert_to_tensor(x_train)

In [30]:
input_layer = tf.keras.layers.Input(shape=dim)
x = tf.keras.layers.Dense(256, activation='relu')(input_layer)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
output_layer = tf.keras.layers.Dense(NB_SPEAKERS, activation='softmax')(x)
model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='model_1')

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 100)]             0         
                                                                 
 dense_37 (Dense)            (None, 256)               25856     
                                                                 
 dropout_31 (Dropout)        (None, 256)               0         
                                                                 
 dense_38 (Dense)            (None, 128)               32896     
                                                                 
 dropout_32 (Dropout)        (None, 128)               0         
                                                                 
 dense_39 (Dense)            (None, 256)               33024     
                                                                 
 dropout_33 (Dropout)        (None, 256)               0   

In [31]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, epochs=100, batch_size=20, validation_split=0.1)

Epoch 1/100
1103/1103 [==============================] - 2s 2ms/step - loss: 5.5453 - accuracy: 0.0067 - val_loss: 5.1954 - val_accuracy: 0.0155
Epoch 2/100
1103/1103 [==============================] - 2s 2ms/step - loss: 4.9121 - accuracy: 0.0306 - val_loss: 3.8179 - val_accuracy: 0.1632
Epoch 3/100
1103/1103 [==============================] - 3s 3ms/step - loss: 2.5834 - accuracy: 0.3604 - val_loss: 0.7360 - val_accuracy: 0.8017
Epoch 4/100
1103/1103 [==============================] - 2s 2ms/step - loss: 0.9074 - accuracy: 0.7430 - val_loss: 0.2437 - val_accuracy: 0.9364
Epoch 5/100
1103/1103 [==============================] - 2s 2ms/step - loss: 0.5212 - accuracy: 0.8470 - val_loss: 0.1331 - val_accuracy: 0.9649
Epoch 6/100
1103/1103 [==============================] - 2s 2ms/step - loss: 0.3893 - accuracy: 0.8841 - val_loss: 0.1040 - val_accuracy: 0.9682
Epoch 7/100
1103/1103 [==============================] - 2s 2ms/step - loss: 0.3386 - accuracy: 0.9014 - val_loss: 0.0819 - val_ac

In [32]:
x_test, id_list = pickle.load(open('dataset-test.p', 'rb'))

x_test = tf.convert_to_tensor(x_test)

In [33]:
stock = model.predict(x_test, verbose=1)

stock = [ np.argmax(f) for f in stock]

127/127 [==============================] - 0s 533us/step


In [34]:
save = [("Id", "Category")]
for i in range(len(stock)):
    save.append((id_list[i], stock[i]))

In [35]:
with open('result.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(save)
